This tutorial covers some of the basic functionality of the sigpyproc package. For a guide on how to extend the
package, see the

## Getting started

For test puproses, few small size filterbank files are included in the `/tests/data/` directory of the `sigpyproc` package.

### Loading data into sigpyproc

Lets start by loading our filterbank file into sigpyproc. To do this, we require the [FilReader](#sigpyproc.readers.FilReader) class from the [sigpyproc.readers](#sigpyproc.readers) module.

In [1]:
import numpy as np
from rich.pretty import Pretty
from sigpyproc.readers import FilReader

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
myFil = FilReader("../../tests/data/tutorial.fil")

In [3]:
myFil

`myFil` now contains an instance of the [FilReader](#sigpyproc.readers.FilReader) class. We can access obervational
metadata through the `myFil.header` attribute:

In [4]:
Pretty(myFil.header)

Header(
    filename='../../tests/data/tutorial.fil',
    data_type='filterbank',
    nchans=64,
    foff=-1.09,
    fch1=1510.0,
    nbits=2,
    tsamp=0.00032,
    tstart=50000.0,
    nsamples=187520,
    nifs=1,
    coord=<SkyCoord (ICRS): (ra, dec) in deg
    (0., 0.)>,
    azimuth=<Angle 0. deg>,
    zenith=<Angle 0. deg>,
    telescope='Fake',
    backend='FAKE',
    source='P: 250.000000000000 ms, DM: 30.000',
    frame='topocentric',
    ibeam=0,
    nbeams=0,
    dm=0,
    period=0,
    accel=0,
    signed=False,
    rawdatafile='',
    stream_info=StreamInfo(
        entries=[
            FileInfo(
                filename='../../tests/data/tutorial.fil',
                hdrlen=244,
                datalen=3000320,
                nsamples=187520,
                tstart=50000.0,
                tsamp=0.00032
            )
        ]
    )
)

where

In [5]:
type(myFil.header)

sigpyproc.header.Header

All values stored in the `myFil.header` attribute may be accessed as attributes, i.e.:

In [6]:
myFil.header.nchans

64

Now that we know how to load a file into `sigpyproc`, let’s look at at doing something with the loaded data.

### Dedispersing the data

One of the most used techniques in pulsar processing is dedispersion, wherein we add or remove frequency dependent
time delays to the data.

To dedisperse our `myFil` instance, we simply call the dedisperse method:

In [7]:
myTim = myFil.dedisperse(30)

Output()

In [8]:
myTim

In [9]:
type(myTim)

sigpyproc.timeseries.TimeSeries

Here we have dedispersed to a DM of 30 pc cm$^{-3}$ with the result being an instance of the [TimeSeries](#sigpyproc.timeseries.TimeSeries) class, which we have called `myTim`.

The [TimeSeries](#sigpyproc.timeseries.TimeSeries) class in a subclass of `numpy.ndarray`, and is capable of using
all standard numpy functions. For example:

In [10]:
myTim.data.sum()

np.float32(19636992.0)

In [11]:
myTim.data.max()

np.float32(121.0)

In [12]:
myTim.data.min()

np.float32(88.0)

In [13]:
np.median(myTim.data)

np.float32(105.0)

The use of `numpy.ndarray` subclasses is important in allowing sigpyproc to easily interface with many 3rd party
python libraries.

### Performing a Fourier transform

To perform a discrete fourier transform of the data contained in the `myTim` instance we may invoke the `myTim.rFFT`
method.

In [14]:
myFS = myTim.rfft()

In [15]:
type(myFS)

sigpyproc.fourierseries.FourierSeries

In [16]:
myFS

The [FourierSeries](#sigpyproc.fourierseries.FourierSeries) is also a subclass of `numpy.ndarray`, where array elements are `numpy.complex64`.

Using the `deredden` method of `myFS`, we can de-redden the Fourier series:

In [17]:
myFS_red = myFS.deredden()

In [18]:
myFS_red

with the dereddened fourier series, we can now form the power spectrum of the observation:

In [19]:
mySpec = myFS_red.form_spec(interpolate=True)

In [20]:
mySpec

Here we have set the `interpolate` flag to True, causing the `formSpec` function to perform nearest bin interpolation.

`mySpec` contains several convenience methods to help with navigating the power spectrum. For instance:

In [21]:
mySpec.period2bin(0.25)

240

In [22]:
mySpec.freq2bin(5.0)

300

We can also perofrm Lyne-Ashworth harmonic folding to an arbitrary number of harmonics:

In [23]:
folds = mySpec.harmonic_fold(5)

In [24]:
folds

Where the variable `folds` is a python list containing each of the requested harmonic folds.

### Folding data

Both the [TimeSeries](#sigpyproc.timeseries.TimeSeries) and the [FourierSeries](#sigpyproc.fourierseries.FourierSeries) 
have methods to phase fold their data. Using our earlier myFil instance, we will fold our filterbank file with a period
of 250 ms and a DM of pc cm$^{-3}$ and acceleration of 0 ms$^{-2}$.

In [25]:
myFold = myFil.fold(0.25,30.,accel=0,nbins=128,nints=32,nbands=16)

Output()

In [26]:
type(myFold)

sigpyproc.foldedcube.FoldedData

In [27]:
myFold.data.shape

(32, 16, 128)

The the [FoldedData](#sigpyproc.foldedcube.FoldedData) has several functions to enable simple slicing and summing of
the folded data cube. These include:

* `get_subband`: select all data in a single frequency band
* `get_subint`: select all data in a single subintegration
* `get_freqPhase`: sum the data in the time axis
* `get_timePhase`: sum the data in the frequency axis
* `get_profile`: get the pulse profile of the fold

We can also tweak the DM and period of the fold using the `update_params` method:

In [28]:
myFold.update_dm(dm=100)
myFold.update_period(period=0.2502)

### Tips and tricks

There are several tips and tricks to help speed up `sigpyproc` and also make it more user friendly. For people who are
familiar with Python and IPython these will be old news, but for newbies these may be of use.

**Tab completion**: One of the many nice things about IPython is that it allows for tab completion:

In [29]:
myFil   # then press tab

**Docstrings**: by using question marks or double question marks we can access both information about a function and
its raw source:

In [30]:
myFil.downsample?

Signature:
myFil.downsample(
    tfactor: 'int' = 1,
    ffactor: 'int' = 1,
    outfile_name: 'str | None' = None,
    gulp: 'int' = 16384,
    start: 'int' = 0,
    nsamps: 'int | None' = None,
    **plan_kwargs: 'Unpack[PlanKwargs]',
) -> 'str'
Docstring:
Downsample data in time and/or frequency and write to file.

Parameters
----------
tfactor : int, optional
    factor by which to downsample in time, by default 1
ffactor : int, optional
    factor by which to downsample in frequency, by default 1
outfile_name : str, optional
    name of file to write to, by default ``basename_tfactor_ffactor.fil``
gulp : int, optional
    number of samples in each read, by default 16384
start : int, optional
    start sample, by default 0
nsamps : int, optional
    number of samples to read, by default all
**plan_kwargs : dict
    Additional keyword arguments for :func:`read_plan`.

Returns
-------
str
    output file name

Raises
------
ValueError
    If number of channels is not divisible by `ff

Note that all docstrings are written in `numpydoc`. This is to facilitate automatic documentation creation with the
Sphinx package.

**Chaining**: The ability to chain together methods, combined with history recall in IPython means that it is simple to
condense a sigpyproc request into a single line:

In [31]:
spectrum = FilReader("../../tests/data/tutorial.fil").collapse().rfft().deredden().form_spec(interpolate=True)

Output()

Here we create a `FilReader` instance, which is then collapsed in frequency, FFT’d, cleaned of rednoise and interpolated to form a power spectrum. In the intrests of readability, this is not always a good idea, however for testing code
quickly, it is invaluable.